This notebook allow you to print results of models in tables

In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
isNotebook = '__file__' not in locals()

In [3]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *

In [4]:
import pandas as pd
from IPython.display import display, HTML

In [6]:
twinewsRankings = getTwinewsRankings()

In [7]:
len(twinewsRankings)

8

In [8]:
def printResults(model=None, splitVersion=None, filt={}, noSubsampling=True, logger=None):
    data = []
    if noSubsampling and "maxUsers" not in filt:
        filt = mergeDicts(filt, {"maxUsers": None})
    if model is not None and "model" not in filt:
        filt = mergeDicts(filt, {"model": model})
    if splitVersion is not None and "splitVersion" not in filt:
        filt = mergeDicts(filt, {"splitVersion": splitVersion})
    for key in twinewsRankings.keys():
        toKeep = True
        meta = twinewsRankings.getMeta(key)
        for filtKey in filt:
            if filtKey not in meta:
                raise Exception(filtKey + "is not in " + b(meta, 5))
            if filt[filtKey] != meta[filtKey]:
                toKeep = False
                break
        if toKeep:
            data.append(meta)
    if len(data) == 0:
        log("No data found", logger)
    else:
        try:
            refKeys = data[0].keys()
            for e in data:
                assert e.keys() == refKeys
        except:
            raise Exception("Some data keys doesn't match:\n" + b(data, 5))
        if len(data) > 1:
            keysHavingSameValues = set(data[0].keys())
            baseValues = data[0]
            for current in data[1:]:
                for key in baseValues.keys():
                    if key in keysHavingSameValues and baseValues[key] != current[key]:
                        keysHavingSameValues.remove(key)
            sameValues = dict()
            for key in keysHavingSameValues:
                sameValues[key] = data[0][key]
            log("These values are common to all rows:\n", logger)
            for key, value in sameValues.items():
                log("\t- " + str(key) + ": " + str(value), logger)
            for i in range(len(data)):
                for key in keysHavingSameValues:
                    del data[i][key]
        df = pd.DataFrame(data)
        display(df)

In [10]:
printResults\
(
    model='nmf',
    splitVersion=2,
    noSubsampling=True,
    filt=\
    {
        # "implementation": "gensim-lda",
    },
)

These values are common to all rows:

	- doLemmatization: True
	- distance: cosine
	- lowercase: True
	- splitVersion: 2
	- nbTopics: 30
	- nmfAlpha: 0.1
	- nmfL1Ratio: 0
	- maxUsers: None
	- nmfInit: nndsvd
	- maxDF: 300
	- useExtraNews: False
	- minDF: 0.0005
	- model: nmf
	- implementation: sklearn-nmf


,maxDocuments,maxIter,historyRef,id
0,1000,200,0.3,nmf-09f9c
1,10000,200,0.3,nmf-dea50
2,1000,100,0.5,nmf-b61b9


In [11]:
printResults\
(
    model='lda',
    splitVersion=2,
    noSubsampling=True,
    filt=\
    {
        # "implementation": "gensim-lda",
    },
)

These values are common to all rows:

	- ldaLearningOffset: 1.0
	- distance: cosine
	- lowercase: True
	- splitVersion: 2
	- ldaLearningMethod: online
	- maxUsers: None
	- ldaLearningDecay: 0.7
	- maxDF: 300
	- useExtraNews: False
	- model: lda


,maxDocuments,minDF,nbTopics,doLemmatization,useTFIDF,maxIter,implementation,historyRef,id
0,10000,0.0005,100,False,True,30,gensim-lda,0.5,lda-b637b
1,1000,0.0005,30,True,True,30,gensim-lda,0.3,lda-aa8d5
2,10000,0.0005,30,True,True,30,gensim-lda,0.3,lda-71511
3,500,0.0020,30,False,False,2,sklearn-lda,0.5,lda-de387
